# Functions

> Some commonly used SQL Functions.

In [ ]:
#| default_exp functions
#| hide
from nbdev.showdoc import *
from fastcore.test import *
# allow multiple output from one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%reload_ext autoreload
%autoreload 2

In [ ]:
#|export
from pikaQ.utils import execute
from pikaQ.terms import FieldBase, Field, custom_func

# Custom Function

In [ ]:
#|export
class CustomFunction(FieldBase):
    def __init__(self, func_name: str, arg_names: list) -> None:
        super().__init__()
        self.func_name = func_name
        self.arg_names = arg_names
        self.get_sql = self.execute
    
    def __call__(self, *args):
        if len(args) != len(self.arg_names):
            raise ValueError(f"The number of args provided {len(args)} is not the same as the number of args expected by this function ({len(self.arg_names)})!")
        def func(*args):
            return f"{self.func_name}({', '.join(args)})"

        self.func = func
        self.args = args
        return self

    def execute(self, **kwargs):
        args = [execute(arg, **kwargs) for arg in self.args]
        return self.func(*args)

`CustomFunction` is a convenient class to create a custom SQL function with the name, and positional arguments, if you don't need the function to be parsed differently for different dialects.

In [ ]:
date_diff = CustomFunction('DATE_DIFF', ['interval', 'start_date', 'end_date'])
test_eq(date_diff('day', 'start_date', 'end_date').get_sql(), 'DATE_DIFF(day, start_date, end_date)')

In [ ]:
date_diff = CustomFunction('DATE_DIFF', ['interval', 'start_date', 'end_date'])
test_eq(date_diff('month', Field('date1'), Field('date2')).get_sql(), 'DATE_DIFF(month, date1, date2)')

## Commonly Used Functions

In [ ]:
#|export
Max = CustomFunction('MAX', ['field'])
Min = CustomFunction('MIN', ['field'])
Sum = CustomFunction('SUM', ['field'])
Avg = CustomFunction('AVG', ['field'])
Count = CustomFunction('COUNT', ['field'])
Abs = CustomFunction('ABS', ['field'])
Round = CustomFunction('ROUND', ['field', 'decimals'])
First = CustomFunction('FIRST', ['field'])
Last = CustomFunction('LAST', ['field'])

@custom_func
def Cast(field, type):
    return f"CAST({field} AS {type})"

@custom_func
def Coalesce(*args):
    return f"COALESCE({', '.join(args)})"

@custom_func
def Concat(*args):
    return f"CONCAT({', '.join(args)})"

def convert_format(format, dialect='sql'):
    if dialect == 'spark':
        return (format.replace('YYYY', 'yyyy')
                .replace('YY', 'yy')
                .replace('DD', 'dd')
                )
    elif dialect == 'snowflake':
        return (format.replace('HH', 'HH24')
                .replace('mm', 'MI')
        )
    else:
        return format

@custom_func
def Date(expression, format=None, dialect='sql'):
    if format:
        format = convert_format(format, dialect)
        return f"DATE('{expression}', '{format}')"
    else:
        return f"DATE('{expression}')"

@custom_func
def AddMonths(date, months, dialect='sql'):
    if dialect in ['snowflake', 'spark']:
        return f"ADD_MONTHS({date}, {months})"
    elif dialect == 'athena':
        return f"date_add('month', {months}, {date})"
    else:
        return f"DATE_ADD({date}, INTERVAL {months} MONTH)"
